### Scenario-1
A retail company receives daily updates for its product catalog, including new products, price changes, and discontinued items. Instead of overwriting the entire catalog or simply appending new records, they need to upset the incoming data - updating existing products with the latest information and inserting new products - ensuring the catalog remains accurate and up-to-date in real-time.

In [0]:
%sql
create schema cat.source;

In [0]:
%sql
create table cat.source.products
(
  id int,
  product_name string,
  price double,
  is_discontinued boolean,
  updated_at timestamp
)
using delta;

In [0]:
%sql
insert into cat.source.products
values
(1, 'productA', 70.0, false, current_timestamp()),
(9, 'productZ', 20.0, false, current_timestamp()),
(3, 'productC', 100.0, false, current_timestamp());

num_affected_rows,num_inserted_rows
3,3


In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [0]:
from delta.tables import DeltaTable

df = spark.read.table("cat.source.products")

df_transform = df.withColumn("dedup", F.row_number().over(Window.partitionBy("id").orderBy(F.desc("updated_at"))))\
                        .filter("dedup = 1")\
                        .drop("dedup")

deltaTable = DeltaTable.forPath(spark, "/Volumes/cat/sink/vol/products_sink")

In [0]:
if len(dbutils.fs.ls("/Volumes/cat/sink/vol/products_sink")) != 0:
    deltaTable.alias("t")\
        .merge(df_transform.alias("s"), "t.id = s.id")\
        .whenMatchedUpdateAll("t.updated_at <= s.updated_at")\
        .whenNotMatchedInsertAll()\
        .execute()

else:
    df_transform.write.format("delta")\
        .mode("overwrite")\
        .save("/Volumes/cat/sink/vol/products_sink")

In [0]:
%sql
select * from delta.`/Volumes/cat/sink/vol/products_sink`;

id,product_name,price,is_discontinued,updated_at
1,product1,30.0,false,2025-08-28T08:05:01.694Z
4,product4,10.0,false,2025-08-28T07:53:52.326Z
6,product2,20.0,false,2025-08-28T08:05:01.694Z
3,product3,30.0,true,2025-08-28T07:53:52.326Z
2,product2,20.0,false,2025-08-28T07:53:52.326Z
5,product5,20.0,false,2025-08-28T07:53:52.326Z


### Scenario-2
A retail company receives daily sales transaction files from multiple store locations in a data lake. Instead of reprocessing all historical data every day, the DE team users spark structured streaming to incrementally load only the newly arrived files into a Delta lake. This ensures timely updates to analytics dashboards while optimizing compute costs and processing time.

In [0]:
order_schema = """
    order_id int,
    customer_id	int,
    order_date date,
    amount double
"""

In [0]:
src = spark.readStream.format("csv")\
                .option("header", True)\
                .schema(order_schema)\
                .load("/Volumes/cat/source/vol/csv_source/")

In [0]:
src.writeStream\
    .format("delta")\
    .option("checkpointLocation", "/Volumes/cat/sink/vol/orders_sink/checkpoint/")\
    .trigger(once=True)\
    .start("/Volumes/cat/sink/vol/orders_sink/data/")

In [0]:
%sql
select * from delta.`/Volumes/cat/sink/vol/orders_sink/data/`;

order_id,customer_id,order_date,amount
6,100,2025-08-07,248.69
7,102,2025-08-08,243.85
8,101,2025-08-09,308.31
9,105,2025-08-10,367.45
10,105,2025-08-11,328.2
1,101,2025-08-02,246.84
2,104,2025-08-03,111.3
3,103,2025-08-04,52.0
4,103,2025-08-05,98.7
5,102,2025-08-06,392.67


### Scenario-3
An e-commerce platform receives customer order details from its mobile application in JSON format through a streaming pipeline. The JSON contains nested fields such as customer information, payment details, and a list of purchased items. To store and analyze this data efficiently in a data warehouse, the nested structure must be flattened into a tabular format using Pyspark, ensuring all relevant attributes are readily accessible for reporting and analytics.

In [0]:
df = spark.read.format("json")\
            .option("multiline", True)\
            .load("/Volumes/cat/source/vol/json_source/")

In [0]:
display(df)

customer,delivery_updates,items,order_id,order_timestamp,payment
"List(CUST101, john.doe@example.com, List(Toronto, Canada), John Doe)","List(Order Placed, Packed, Shipped, Out for Delivery)","List(List(ITEM1001, 25.5, Wireless Mouse, 2), List(ITEM1002, 199.75, Mechanical Keyboard, 1))",ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)"
"List(CUST102, jane.smith@example.com, List(Vancouver, Canada), Jane Smith)","List(Order Placed, Packed, Shipped)","List(List(ITEM1003, 89.99, USB-C Hub, 1))",ORD002,2025-08-15T11:10:15Z,"List(89.99, CAD, PayPal)"


In [0]:
df_transform = df.withColumn("delivery_updates", F.explode(F.col("delivery_updates")))\
                 .withColumn("items", F.explode(F.col("items")))

df_transform = df_transform.select("customer.customer_id", "customer.email", "customer.location.city", "customer.location.country", "customer.name", "delivery_updates", "items.item_id", "items.price_per_unit", "items.product_name", "items.quantity", "order_id", "order_timestamp", "payment.amount", "payment.currency", "payment.method")

In [0]:
df_transform.createOrReplaceTempView("orders")

spark.sql(f"""
          create table cat.sink.orders
          select * from orders
          """)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from cat.sink.orders;

customer_id,email,city,country,name,delivery_updates,item_id,price_per_unit,product_name,quantity,order_id,order_timestamp,amount,currency,method
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Order Placed,ITEM1001,25.5,Wireless Mouse,2,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Order Placed,ITEM1002,199.75,Mechanical Keyboard,1,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Packed,ITEM1001,25.5,Wireless Mouse,2,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Packed,ITEM1002,199.75,Mechanical Keyboard,1,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Shipped,ITEM1001,25.5,Wireless Mouse,2,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Shipped,ITEM1002,199.75,Mechanical Keyboard,1,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Out for Delivery,ITEM1001,25.5,Wireless Mouse,2,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Toronto,Canada,John Doe,Out for Delivery,ITEM1002,199.75,Mechanical Keyboard,1,ORD001,2025-08-15T10:45:30Z,250.75,CAD,Credit Card
CUST102,jane.smith@example.com,Vancouver,Canada,Jane Smith,Order Placed,ITEM1003,89.99,USB-C Hub,1,ORD002,2025-08-15T11:10:15Z,89.99,CAD,PayPal
CUST102,jane.smith@example.com,Vancouver,Canada,Jane Smith,Packed,ITEM1003,89.99,USB-C Hub,1,ORD002,2025-08-15T11:10:15Z,89.99,CAD,PayPal


### Scenario-4
In a large scale data processing project, multiple Pyspark notebooks require the same set of custom transformation functions, such as data formatting, null handling, and data validation. Instead of duplicating the code across notebooks, a Python class is created to store these reusable functions. This ensures consistency, reduces maintenance effort, and improves code readability across the project.

In [0]:
class Transformation():

  def __init__(self, df):
    self.df = df

  def dedup(self, id_key, order_key):
    self.df = self.df.withColumn("dedup", F.row_number().over(Window.partitionBy(id_key).orderBy(F.desc(order_key))))\
                     .filter("dedup = 1")\
                     .drop("dedup")
    return self.df

  def remove_null(self, column):
    self.df = self.df.filter(F.col(column).isNotNull())
    return self.df


In [0]:
df = spark.read.table("cat.source.products")
display(df)

id,product_name,price,is_discontinued,updated_at
1,product1,10.0,false,2025-08-28T07:53:52.326Z
2,product2,20.0,false,2025-08-28T07:53:52.326Z
3,product3,30.0,true,2025-08-28T07:53:52.326Z
4,product4,10.0,false,2025-08-28T07:53:52.326Z
5,product5,20.0,false,2025-08-28T07:53:52.326Z
1,product1,30.0,false,2025-08-28T08:04:41.512Z
6,product2,20.0,false,2025-08-28T08:04:41.512Z
1,product1,30.0,false,2025-08-28T08:05:01.694Z
6,product2,20.0,false,2025-08-28T08:05:01.694Z


In [0]:
tobj = Transformation(df)

df = tobj.dedup("id", "updated_at")
df = tobj.remove_null("updated_at")

display(df)

id,product_name,price,is_discontinued,updated_at
1,product1,30.0,false,2025-08-28T08:05:01.694Z
2,product2,20.0,false,2025-08-28T07:53:52.326Z
3,product3,30.0,true,2025-08-28T07:53:52.326Z
4,product4,10.0,false,2025-08-28T07:53:52.326Z
5,product5,20.0,false,2025-08-28T07:53:52.326Z
6,product2,20.0,false,2025-08-28T08:05:01.694Z


### Scenario-5
A retail company maintains a product catalog in its data warehouse. Product details such as name, category, and price may change over time due to rebranding, category updates, or pricing adjustments. To preserve historical data for accurate reporting and trend analysis, the company needs to implement a slowly changing dimension (SCD) type 2 mechanism in Pyspark, ensuring old records are retained with effective data ranges while new versions are inserted as separate records.

In [0]:
df = spark.read.table("cat.source.products")

tobj = Transformation(df)
df_transformed = tobj.dedup("id", "updated_at")

In [0]:
df_transformed.createOrReplaceTempView("products")

if spark.catalog.tableExists("cat.sink.dim_products"):
    spark.sql("""
              merge into cat.sink.dim_products as dst
              using products as src
              on dst.id = src.id and dst.is_current = true
              and dst.updated_at > src.updated_at
              when matched and (
                dst.product_name != src.product_name or
                dst.price != src.price or
                dst.is_discontinued != src.is_discontinued or
                dst.updated_at != src.updated_at
              )
              then update set 
              is_current = false,
              end_date = current_date();
              """)
    
    spark.sql("""
              merge into cat.sink.dim_products as dst
              using products as src
              on dst.id = src.id and dst.is_current = true
              when not matched then insert (
                id,
                product_name,
                price,
                is_discontinued,
                updated_at,
                start_date,
                end_date,
                is_current
              ) values (
                src.id,
                src.product_name,
                src.price,
                src.is_discontinued,
                src.updated_at,
                current_date(),
                '9999-01-01',
                true
              );
              """)
else:
    spark.sql("""
              create table cat.sink.dim_products
              select
                id,
                product_name,
                price,
                is_discontinued,
                updated_at,
                current_date() as start_date,
                '9999-01-01' as end_date,
                true as is_current
              from products
              """)

In [0]:
%sql
select * from cat.sink.dim_products;

id,product_name,price,is_discontinued,updated_at,start_date,end_date,is_current
2,product2,20.0,false,2025-08-28T07:53:52.326Z,2025-08-28,9999-01-01,true
4,product4,10.0,false,2025-08-28T07:53:52.326Z,2025-08-28,9999-01-01,true
5,product5,20.0,false,2025-08-28T07:53:52.326Z,2025-08-28,9999-01-01,true
6,product2,20.0,false,2025-08-28T08:05:01.694Z,2025-08-28,9999-01-01,true
1,product1,30.0,false,2025-08-28T08:05:01.694Z,2025-08-28,2025-08-28,false
3,product3,30.0,true,2025-08-28T07:53:52.326Z,2025-08-28,2025-08-28,false
1,productA,70.0,false,2025-08-28T09:33:40.013Z,2025-08-28,9999-01-01,true
3,productC,100.0,false,2025-08-28T09:33:40.013Z,2025-08-28,9999-01-01,true
9,productZ,20.0,false,2025-08-28T09:33:40.013Z,2025-08-28,9999-01-01,true


### Scenario-6
A retail company processes daily sales transactions from multiple store locations. The data arrives in different formats and needs to be cleaned, validated, and aggregated for business reporting. Using Delta Live Tables pipeline, the raw data is ingested from cloud storage, transformed with quality check, and stored in Delta tables for analytics dashboards, ensuring accuracy and reliability in near real-time.

In [0]:
import dlt
from pyspark.sql.window import Window
import pyspark.sql.functions as F

@dlt.table(name="customers_source")
def customers_source():
  df = spark.read.table("cat.source.customers")
  return df

@dlt.table(name="customers_transformed")
def customers_transformed():
  df = spark.read.table("customers_source")
  df = df.withColumn("dedup", F.row_number().over(Window.partitionBy("id").orderBy(F.desc("updated_at"))))\
        .filter("dedup == 1")\
        .drop("dedup")
  return df

dlt.create_streaming_table("dim_customers")

dlt.create_auto_cdc_flow(
  target = "dim_customers",
  source = "customers_transformed",
  keys = ["id"],
  sequence_by = "updated_at",
  ignore_null_updates = False,
  apply_as_deletes = None,
  apply_as_truncates = None,
  column_list = None,
  except_column_list = None,
  stored_as_scd_type = 2,
  track_history_column_list = None,
  track_history_except_column_list = None,
  name = None,
  once = False
)
